# Banalisation de données
Une fonction de banalisation permet d'extraire des données internes et de les convertir dans un format HDF5 avec des mesures banalisées.

In [5]:
# from importlib import reload
import os
from tabata import etdex  # Pour charger un itérateur sur des données.
from tabata import banalise # Je ne sais pas commen simplifier ça !

Je mets souvent une commande reload pour vérifier la mise au point de mes codes.

In [6]:
%reload_ext autoreload
%autoreload 2

### Fonction banalise()
Cette fonction admet les arguments suivants :

* _dfiterator_ est un itérateur qui distribue des DataFrames pandas.
* _storename_ est le nom (chemin complet) du fichiers HDF5 de stockage.
* _nbmax_ (optionnel) permet de limiter le nombre d'enregistrements à sauvegarder, None par défaut pour tout scanner.
* _delay_ (optionnel) donne le délai retranché aux index.Par défaut il est fixé à 11 ans et 2 mois et 3 jours, 2 heures et 12 minutes.
* _iqr_ (optionnel) est la valeur des inter-quantiles utilisés, par défaut iqr=0.25 soit [0.25,0.75].
* _nanvalues_ (optionnel) spécifie les valeurs à remplacer par des NAN, par défaut nanvalues=[-999.0] pour être compatible avec la base MOISE. 

Cette fonction transforme les DataFrames avant de les recopier suivant la procédure suivante :
* (1) Supprime les colonnes toujours NAN ou toujours constantes        (remplace aussi les valeurs -999 par des NAN).
* (2) Recrée des unités propres.
* (3) Calcule les maximums et minimums des interquantiles des données par unité.
* (4) Par unité le facteur multiplicatif appliqué est la partie entière du max des interquantiles divisée par l'ordre de grandeur en base 10.
        
Crée aussi un fichier LOG contenant la date de banalisation et les opérations effectuées avec un facteur multiplicatif par unité et un décalage temporel global. La date de l'opération est concaténée entre parenthèse à la fin des noms des fichiers H5 et LOG.

Un fichier d'alias composé de labels suivi d'un alias banalisé peut être exploité s'il est fourni. Dans le cas contraire les labels initiaux sont conservés.

Ce fichier sera concaténé à la fin du fichier de log.

In [7]:
dirbancs = os.path.expanduser('~/wrk/bancs')
aliasfile = os.path.normpath(os.path.join(dirbancs,'extract/alias.txt'))
if os.path.isfile(aliasfile):
    with open(aliasfile,'r') as f:
        txt = f.read()
    print(txt[:300],'...')

ARINCTIM00  	TS          
Correcte00  	C00         
COWL_FLO00  	COWL0       
COWL_FLO01  	COWL1       
CR25A55C02  	CR25A55C02  
CR25A55C15  	CR25A55C15  
CR25A55C32  	CR25A55C32  
CR26055C02  	CR26055C02  
CR26055C15  	C15         
CR26055C32  	CR26055C32  
CR26B55C14  	CR26B55C14  
CR26B55C29  	C ...


Un itérateur sur des DataFrames doit être fourni. Dans cet exemple on travaille sur un répertoire de fichiers ETDEX issus de MOISE.

Un commentaire peut être rajouté en entête du fichier de Log qui rappelle le traitement effectué.

In [8]:
dirname = os.path.join(dirbancs,"data/sam146")
iterator = etdex.dataiterator(dirname)

bananame = os.path.join(dirbancs,'extract/banal.h5')
banalise(iterator,bananame,nbmax=10,
                   aliasfile=aliasfile,
                   comment="SAM146, année2007")

In [19]:
dirbancs

'H:\\/wrk/bancs'

In [20]:
!date /T
!time /T
!dir H:\wrk\bancs\extract\banal*.*

06/10/2020 
14:19
 Le volume dans le lecteur H s'appelle Windows
 Le num‚ro de s‚rie du volume est 084B-F5B9

 R‚pertoire de H:\wrk\bancs\extract

06/10/2020  14:15         1ÿ269ÿ752 banal.h5
06/10/2020  14:15             7ÿ174 banal.log
               2 fichier(s)        1ÿ276ÿ926 octets
               0 R‚p(s)  63ÿ200ÿ129ÿ024 octets libres


In [21]:
!type C:\Users\s068990\Documents\HOME\wrk\bancs\extract\banal*.log

SAM146, année2007


C:\Users\s068990\Documents\HOME\wrk\bancs\extract\banal.log






Date de la banalisation      : 2020-10-06
Fichier de sauvegarde hdf5   : H:\/wrk/bancs\extract/banal.h5
Fichier de log               : H:\/wrk/bancs\extract/banal.log

Nombre d'enregistrements     : 10
Nombre de labels             : 37
Nb de labels non constants   : 37
Nombre d'unités distinctes   : 9

Facteurs de correction par label
                           m : 0.50
                           K : 3.07
                           - : 1.92
                         m/s : 0.56
                      kg/cm2 : 3.00
                        kg/s : 2.00
                           s : 2.00
                           V : 2.19
                           C : 2.10
Enregistrements :
record_0    	 BCIAM_C1A__TR__DEV__d20070907_h124025__17
record_1    	 BPOL_OATB__TR__DEV__d20070601_h121558__100
record_2    	 BPOL_OATB__TR__DEV__d20070601_h124523__100
record_3    	 BPOL_OATB__TR__DEV__d20070601_h124526__5
record_4    	 BPOL_OATB__TR__DEV__d20070601_h170205__5
record_5    	 BPOL_OATB__TR__DEV__d2007

_Jérôme Lacaille (YXE)_